In [1]:
from Input_Variables.read_vars import raw_data_storage, \
                                      daily_stats_features_lower, daily_stats_features_upper, \
                                      ml_models_train_split, ml_models_test_split

from Data_Schema.schema import Pandas_UDF_Data_Schema
from Read_In_Data.read_data import Reading_Data
from Data_Pipeline.sklearn_pipeline import Sklearn_Pipeline
from Feature_Generation.create_binary_labels import Create_Binary_Labels
from Feature_Generation.summary_stats import Summary_Stats_Features
from Models.xgboost_model import XGBoost_Classification

# General Modules

In [2]:
# PySpark UDF Schema Activation
pandas_udf_data_schema=Pandas_UDF_Data_Schema()

# Data Location
reading_data=Reading_Data(data_location=raw_data_storage)

# Create Binary y Variables
create_binary_labels=Create_Binary_Labels()

# Sklearn Pipeline
pandas_sklearn_pipeline=Sklearn_Pipeline()

# Features Daily Stats Module
summary_stats_features=Summary_Stats_Features()

# XGBoost Model Module
xgboost_classification=XGBoost_Classification()


# PySpark

### PySpark: Reading In Data

In [3]:
####### PySpark
pyspark_df=reading_data.read_in_pyspark()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/01 17:29:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### PySpark: Custom Imputation Pipeline

In [4]:
####### PySpark
pyspark_custom_imputation_schema=pandas_udf_data_schema.custom_imputation_pyspark_schema()
pyspark_custom_imputation_pipeline=pandas_sklearn_pipeline.pyspark_custom_imputation_pipeline(df=pyspark_df, 
                                                                                              output_schema=pyspark_custom_imputation_schema)
pyspark_custom_imputation_pipeline.show(1)

/Users/carlosmonsivais/Desktop/glucose-data-analysis/glucose_venv/lib/python3.10/site-packages/pyspark/sql/pandas/group_ops.py:98: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


23/03/01 17:29:17 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
23/03/01 17:29:18 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: PostDate, IngestionDate, PostId, PostTime, PatientId, Stream, SequenceNumber, TransmitterNumber, ReceiverNumber, RecordedSystemTime, RecordedDisplayTime, RecordedDisplayTimeRaw, TransmitterId, TransmitterTime, GlucoseSystemTime, GlucoseDisplayTime, GlucoseDisplayTimeRaw, Value, Status, TrendArrow, TrendRate, IsBackFilled, InternalStatus, SessionStartTime
 Schema: PostDate, IngestionDate, PostID, PostTime, PatientId, Stram, SequenceNumber, TransmitterNumber, ReceiverNumber, RecordedSystemTime, RecordedDisplayTime, RecordedDisplayTimeRaw, TransmitterId, TransmitterTime, GlucoseSystemTime, GlucoseDisplayTime, GlucoseDisplayTimeRaw, Value, Status, TrendArrow, TrendRate, IsBackFilled, InternalStatus, SessionStartTime
Expec

+--------------------+-----+-------------------+------------------+--------+-------+
|           PatientId|Value| GlucoseDisplayTime|GlucoseDisplayDate|inserted|missing|
+--------------------+-----+-------------------+------------------+--------+-------+
|tHu8WPnIffml5CL+A...|111.0|2022-09-13 19:15:45|        2022-09-13|       0|      0|
+--------------------+-----+-------------------+------------------+--------+-------+
only showing top 1 row



/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['inserted'] = 0
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['missing'].iloc[missing_label] = 1
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

### PySpark Aggregate Data at Level

### PySpark: Adding Binary Labels

In [5]:
pyspark_df_added_binary_labels=create_binary_labels.pyspark_binary_labels(df=pyspark_custom_imputation_pipeline, 
                                                                          lower=daily_stats_features_lower, 
                                                                          upper=daily_stats_features_upper)

pyspark_df_added_binary_labels.show(1)

23/03/01 17:29:20 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: PostDate, IngestionDate, PostId, PostTime, PatientId, Stream, SequenceNumber, TransmitterNumber, ReceiverNumber, RecordedSystemTime, RecordedDisplayTime, RecordedDisplayTimeRaw, TransmitterId, TransmitterTime, GlucoseSystemTime, GlucoseDisplayTime, GlucoseDisplayTimeRaw, Value, Status, TrendArrow, TrendRate, IsBackFilled, InternalStatus, SessionStartTime
 Schema: PostDate, IngestionDate, PostID, PostTime, PatientId, Stram, SequenceNumber, TransmitterNumber, ReceiverNumber, RecordedSystemTime, RecordedDisplayTime, RecordedDisplayTimeRaw, TransmitterId, TransmitterTime, GlucoseSystemTime, GlucoseDisplayTime, GlucoseDisplayTimeRaw, Value, Status, TrendArrow, TrendRate, IsBackFilled, InternalStatus, SessionStartTime
Expected: Stram but found: Stream
CSV file: file:///Users/carlosmonsivais/Desktop/capstone_data/ahr414_glucose_sample%20-%20ahr414_glucose_sample.csv


+--------------------+-----+-------------------+------------------+--------+-------+--------+
|           PatientId|Value| GlucoseDisplayTime|GlucoseDisplayDate|inserted|missing|y_Binary|
+--------------------+-----+-------------------+------------------+--------+-------+--------+
|tHu8WPnIffml5CL+A...|111.0|2022-09-13 19:15:45|        2022-09-13|       0|      0|       0|
+--------------------+-----+-------------------+------------------+--------+-------+--------+
only showing top 1 row



/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['inserted'] = 0
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['missing'].iloc[missing_label] = 1
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

### PySpark: Features: Summary Statistics

In [6]:
####### PySpark
pyspark_features_summary_stats_schema=pandas_udf_data_schema.summary_stats_schema()
pyspark_features_summary_stats=summary_stats_features.pyspark_summary_statistics(df=pyspark_df_added_binary_labels, 
                                                                                 output_schema=pyspark_features_summary_stats_schema, 
                                                                                 lower=daily_stats_features_lower, 
                                                                                 upper=daily_stats_features_upper)

pyspark_features_summary_stats.show(1)

23/03/01 17:29:21 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: PostDate, IngestionDate, PostId, PostTime, PatientId, Stream, SequenceNumber, TransmitterNumber, ReceiverNumber, RecordedSystemTime, RecordedDisplayTime, RecordedDisplayTimeRaw, TransmitterId, TransmitterTime, GlucoseSystemTime, GlucoseDisplayTime, GlucoseDisplayTimeRaw, Value, Status, TrendArrow, TrendRate, IsBackFilled, InternalStatus, SessionStartTime
 Schema: PostDate, IngestionDate, PostID, PostTime, PatientId, Stram, SequenceNumber, TransmitterNumber, ReceiverNumber, RecordedSystemTime, RecordedDisplayTime, RecordedDisplayTimeRaw, TransmitterId, TransmitterTime, GlucoseSystemTime, GlucoseDisplayTime, GlucoseDisplayTimeRaw, Value, Status, TrendArrow, TrendRate, IsBackFilled, InternalStatus, SessionStartTime
Expected: Stram but found: Stream
CSV file: file:///Users/carlosmonsivais/Desktop/capstone_data/ahr414_glucose_sample%20-%20ahr414_glucose_sample.csv


+--------------------+-----+-------------------+------------------+--------+-------+--------+------+---------+--------+-----+-----+---------+---------+
|           PatientId|Value| GlucoseDisplayTime|GlucoseDisplayDate|inserted|missing|y_Binary|Median|     Mean| Std Dev|  Max|  Min|AreaBelow|AreaAbove|
+--------------------+-----+-------------------+------------------+--------+-------+--------+------+---------+--------+-----+-----+---------+---------+
|tHu8WPnIffml5CL+A...|111.0|2022-09-13 19:15:45|        2022-09-13|       0|      0|       0| 204.0|198.17545|44.67156|256.0|109.0|      0.0|   1697.0|
+--------------------+-----+-------------------+------------------+--------+-------+--------+------+---------+--------+-----+-----+---------+---------+
only showing top 1 row



/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['inserted'] = 0
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['missing'].iloc[missing_label] = 1
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

### PySpark: Wrapper Functions

### PySpark: Sklearn Categorical Pipeline in PySpark

In [7]:
####### PySpark
pyspark_categorical_schema=pandas_udf_data_schema.sklearn_pyspark_categorical_schema()
pyspark_transformations_categorical=pandas_sklearn_pipeline.pyspark_sklearn_pipeline_categorical(df=pyspark_features_summary_stats, 
                                                                                                 output_schema=pyspark_categorical_schema)
pyspark_transformations_categorical.show(1)

23/03/01 17:29:22 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: PostDate, IngestionDate, PostId, PostTime, PatientId, Stream, SequenceNumber, TransmitterNumber, ReceiverNumber, RecordedSystemTime, RecordedDisplayTime, RecordedDisplayTimeRaw, TransmitterId, TransmitterTime, GlucoseSystemTime, GlucoseDisplayTime, GlucoseDisplayTimeRaw, Value, Status, TrendArrow, TrendRate, IsBackFilled, InternalStatus, SessionStartTime
 Schema: PostDate, IngestionDate, PostID, PostTime, PatientId, Stram, SequenceNumber, TransmitterNumber, ReceiverNumber, RecordedSystemTime, RecordedDisplayTime, RecordedDisplayTimeRaw, TransmitterId, TransmitterTime, GlucoseSystemTime, GlucoseDisplayTime, GlucoseDisplayTimeRaw, Value, Status, TrendArrow, TrendRate, IsBackFilled, InternalStatus, SessionStartTime
Expected: Stram but found: Stream
CSV file: file:///Users/carlosmonsivais/Desktop/capstone_data/ahr414_glucose_sample%20-%20ahr414_glucose_sample.csv


/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['inserted'] = 0
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['missing'].iloc[missing_label] = 1
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

+--------------------+-----+-------------------+------------------+--------+------+---------+--------+-----+-----+---------+---------+--------+-------+
|           PatientId|Value| GlucoseDisplayTime|GlucoseDisplayDate|y_Binary|Median|     Mean| Std Dev|  Max|  Min|AreaBelow|AreaAbove|inserted|missing|
+--------------------+-----+-------------------+------------------+--------+------+---------+--------+-----+-----+---------+---------+--------+-------+
|tHu8WPnIffml5CL+A...|111.0|2022-09-13 19:15:45|        2022-09-13|       0| 204.0|198.17545|44.67156|256.0|109.0|      0.0|   1697.0|  [1, 0]| [1, 0]|
+--------------------+-----+-------------------+------------------+--------+------+---------+--------+-----+-----+---------+---------+--------+-------+
only showing top 1 row



### PySpark: Sklearn Numerical Pipeline in PySpark

In [8]:
####### PySpark
pyspark_numerical_schema=pandas_udf_data_schema.sklearn_pyspark_numerical_schema()
pyspark_transformations_numerical=pandas_sklearn_pipeline.pyspark_sklearn_pipeline_numerical(df=pyspark_transformations_categorical, 
                                                                                             output_schema=pyspark_numerical_schema)
pyspark_transformations_numerical.show(1)

23/03/01 17:29:24 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: PostDate, IngestionDate, PostId, PostTime, PatientId, Stream, SequenceNumber, TransmitterNumber, ReceiverNumber, RecordedSystemTime, RecordedDisplayTime, RecordedDisplayTimeRaw, TransmitterId, TransmitterTime, GlucoseSystemTime, GlucoseDisplayTime, GlucoseDisplayTimeRaw, Value, Status, TrendArrow, TrendRate, IsBackFilled, InternalStatus, SessionStartTime
 Schema: PostDate, IngestionDate, PostID, PostTime, PatientId, Stram, SequenceNumber, TransmitterNumber, ReceiverNumber, RecordedSystemTime, RecordedDisplayTime, RecordedDisplayTimeRaw, TransmitterId, TransmitterTime, GlucoseSystemTime, GlucoseDisplayTime, GlucoseDisplayTimeRaw, Value, Status, TrendArrow, TrendRate, IsBackFilled, InternalStatus, SessionStartTime
Expected: Stram but found: Stream
CSV file: file:///Users/carlosmonsivais/Desktop/capstone_data/ahr414_glucose_sample%20-%20ahr414_glucose_sample.csv


/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['inserted'] = 0
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['missing'].iloc[missing_label] = 1
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

+--------------------+-----------+-------------------+------------------+---------+----------+-----------+----------+---------+----------+-----------+--------+-------+--------+
|           PatientId|      Value| GlucoseDisplayTime|GlucoseDisplayDate|   Median|      Mean|    Std Dev|       Max|      Min| AreaBelow|  AreaAbove|inserted|missing|y_Binary|
+--------------------+-----------+-------------------+------------------+---------+----------+-----------+----------+---------+----------+-----------+--------+-------+--------+
|tHu8WPnIffml5CL+A...|-0.85579604|2022-09-13 19:15:45|        2022-09-13|0.7810994|0.44205618|-0.21006547|-0.3378761|0.2710541|-0.5248147|-0.79469234|  [1, 0]| [1, 0]|       0|
+--------------------+-----------+-------------------+------------------+---------+----------+-----------+----------+---------+----------+-----------+--------+-------+--------+
only showing top 1 row



### PySpark: XGBoost Model

In [9]:
####### PySpark
pyspark_xgboost_schema=pandas_udf_data_schema.XGBoost_schema()

test_model1=xgboost_classification.pyspark_xgboost(df=pyspark_transformations_numerical, 
                                                   output_schema=pyspark_xgboost_schema, 
                                                   train_split=ml_models_train_split, 
                                                   test_split=ml_models_test_split)
test_model1.show()

23/03/01 17:29:25 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: PostDate, IngestionDate, PostId, PostTime, PatientId, Stream, SequenceNumber, TransmitterNumber, ReceiverNumber, RecordedSystemTime, RecordedDisplayTime, RecordedDisplayTimeRaw, TransmitterId, TransmitterTime, GlucoseSystemTime, GlucoseDisplayTime, GlucoseDisplayTimeRaw, Value, Status, TrendArrow, TrendRate, IsBackFilled, InternalStatus, SessionStartTime
 Schema: PostDate, IngestionDate, PostID, PostTime, PatientId, Stram, SequenceNumber, TransmitterNumber, ReceiverNumber, RecordedSystemTime, RecordedDisplayTime, RecordedDisplayTimeRaw, TransmitterId, TransmitterTime, GlucoseSystemTime, GlucoseDisplayTime, GlucoseDisplayTimeRaw, Value, Status, TrendArrow, TrendRate, IsBackFilled, InternalStatus, SessionStartTime
Expected: Stram but found: Stream
CSV file: file:///Users/carlosmonsivais/Desktop/capstone_data/ahr414_glucose_sample%20-%20ahr414_glucose_sample.csv


/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['inserted'] = 0
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['missing'].iloc[missing_label] = 1
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

23/03/01 17:29:26 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: PostDate, IngestionDate, PostId, PostTime, PatientId, Stream, SequenceNumber, TransmitterNumber, ReceiverNumber, RecordedSystemTime, RecordedDisplayTime, RecordedDisplayTimeRaw, TransmitterId, TransmitterTime, GlucoseSystemTime, GlucoseDisplayTime, GlucoseDisplayTimeRaw, Value, Status, TrendArrow, TrendRate, IsBackFilled, InternalStatus, SessionStartTime
 Schema: PostDate, IngestionDate, PostID, PostTime, PatientId, Stram, SequenceNumber, TransmitterNumber, ReceiverNumber, RecordedSystemTime, RecordedDisplayTime, RecordedDisplayTimeRaw, TransmitterId, TransmitterTime, GlucoseSystemTime, GlucoseDisplayTime, GlucoseDisplayTimeRaw, Value, Status, TrendArrow, TrendRate, IsBackFilled, InternalStatus, SessionStartTime
Expected: Stram but found: Stream
CSV file: file:///Users/carlosmonsivais/Desktop/capstone_data/ahr414_glucose_sample%20-%20ahr414_glucose_sample.csv


/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['inserted'] = 0
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['missing'].iloc[missing_label] = 1
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

+--------------------+-------------------+-----------+------+
|           PatientId| GlucoseDisplayTime|Predictions|Actual|
+--------------------+-------------------+-----------+------+
|9Fob2bu+MwNd3P522...|2022-09-13 19:39:18|          1|     1|
|9Fob2bu+MwNd3P522...|2022-09-13 19:44:18|          1|     1|
|9Fob2bu+MwNd3P522...|2022-09-13 19:49:18|          0|     0|
|9Fob2bu+MwNd3P522...|2022-09-13 19:54:19|          0|     0|
|9Fob2bu+MwNd3P522...|2022-09-13 19:59:18|          1|     1|
|9Fob2bu+MwNd3P522...|2022-09-13 20:04:18|          1|     1|
|9Fob2bu+MwNd3P522...|2022-09-13 20:09:18|          1|     1|
|9Fob2bu+MwNd3P522...|2022-09-13 20:14:18|          1|     1|
|9Fob2bu+MwNd3P522...|2022-09-13 20:19:19|          1|     1|
|9Fob2bu+MwNd3P522...|2022-09-13 20:24:18|          1|     1|
|9Fob2bu+MwNd3P522...|2022-09-13 20:29:19|          0|     0|
|9Fob2bu+MwNd3P522...|2022-09-13 20:34:18|          1|     1|
|9Fob2bu+MwNd3P522...|2022-09-13 20:39:19|          1|     1|
|9Fob2bu

# Pandas

### Pandas: Reading in Data

In [10]:
####### Pandas
pandas_df=reading_data.read_in_pandas()
pandas_df.head(1)

,PostDate,IngestionDate,PostId,PostTime,PatientId,Stream,SequenceNumber,TransmitterNumber,ReceiverNumber,RecordedSystemTime,...,GlucoseSystemTime,GlucoseDisplayTime,GlucoseDisplayTimeRaw,Value,Status,TrendArrow,TrendRate,IsBackFilled,InternalStatus,SessionStartTime
0,2022-09-14,2022-09-14,fyQ0wOxwB8sthzC75TWKpqdrCMCLq+W1wBg9s7MHVcE=,2022-09-14 00:26:00,tHu8WPnIffml5CL+AbOBkXcbFApQnP06KdrHbjinta4=,Phone7,1663115129,XFgG633aV9tw5Gclf1WB6nmt9Wgv4nHJjXfRQIAOfsg=,NaN,2022-09-13 23:16:01,...,2022-09-13 23:15:45,2022-09-13 19:15:45,2022-09-13 19:15:45-04:00,111.0,NaN,Flat,0.2,FALSE,6,5137335


### Pandas: Custom Imputation Pipeline

In [11]:
####### Pandas
pandas_custom_imputation_pipeline=pandas_sklearn_pipeline.pandas_custom_imputation_pipeline(df=pandas_df)
pandas_custom_imputation_pipeline.head(1)

/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['inserted'] = 0
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['missing'].iloc[missing_label] = 1
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

,PatientId,Value,GlucoseDisplayTime,GlucoseDisplayDate,inserted,missing
0,tHu8WPnIffml5CL+AbOBkXcbFApQnP06KdrHbjinta4=,111.0,2022-09-13 19:15:45,2022-09-13,0,0.0


### Pandas: Aggregate Data at Level

### Pandas: Adding Binary Labels

In [12]:
pandas_df_added_binary_labels=create_binary_labels.pandas_binary_labels(df=pandas_custom_imputation_pipeline, 
                                                                        lower=daily_stats_features_lower, 
                                                                        upper=daily_stats_features_upper)
pandas_df_added_binary_labels.head(1)

,PatientId,Value,GlucoseDisplayTime,GlucoseDisplayDate,inserted,missing,y_Binary
0,tHu8WPnIffml5CL+AbOBkXcbFApQnP06KdrHbjinta4=,111.0,2022-09-13 19:15:45,2022-09-13,0,0.0,0


### Pandas: Features: Summary Statistics

In [13]:
pandas_features_summary_stats=summary_stats_features.pandas_compressDailyValues(data=pandas_df_added_binary_labels, 
                                                                                lower=daily_stats_features_lower, 
                                                                                upper=daily_stats_features_upper)
pandas_features_summary_stats.head(1)

,PatientId,Value,GlucoseDisplayTime,GlucoseDisplayDate,y_Binary,inserted,missing,Median,Mean,Std Dev,Max,Min,AreaBelow,AreaAbove
0,tHu8WPnIffml5CL+AbOBkXcbFApQnP06KdrHbjinta4=,111.0,2022-09-13 19:15:45,2022-09-13,0,0,0.0,204.0,198.175439,44.671559,256.0,109.0,0.0,1697.0


### Pandas: Wrapper Functions

### Pandas: Sklearn Categorical Pipeline in Pandas

In [14]:
####### Pandas
pandas_custom_categorical_pipeline=pandas_sklearn_pipeline.pandas_transform_categorical_features(df=pandas_features_summary_stats)
pandas_custom_categorical_pipeline.head(1)

,PatientId,Value,GlucoseDisplayTime,GlucoseDisplayDate,Median,Mean,Std Dev,Max,Min,AreaBelow,AreaAbove,inserted,missing,y_Binary
0,tHu8WPnIffml5CL+AbOBkXcbFApQnP06KdrHbjinta4=,111.0,2022-09-13 19:15:45,2022-09-13,204.0,198.175439,44.671559,256.0,109.0,0.0,1697.0,"[1.0, 0.0]","[1.0, 0.0]",0


### Pandas: Sklearn Numerical Pipeline in Pandas

In [15]:
####### Pandas
pandas_custom_numerical_pipeline=pandas_sklearn_pipeline.pandas_transform_numerical_features(df=pandas_custom_categorical_pipeline)
pandas_custom_numerical_pipeline.head(1)

,PatientId,Value,GlucoseDisplayTime,GlucoseDisplayDate,Median,Mean,Std Dev,Max,Min,AreaBelow,AreaAbove,inserted,missing,y_Binary
0,tHu8WPnIffml5CL+AbOBkXcbFApQnP06KdrHbjinta4=,-0.855005,2022-09-13 19:15:45,2022-09-13,0.781099,0.438937,-0.214609,-0.339527,0.271054,-0.520029,-0.794176,"[1.0, 0.0]","[1.0, 0.0]",0


### Pandas: Models